## Judging Criteria for the *Synthetic* Track

The following (measurable and objective) aspects are considered to evaluate the quality of an SR method:

* `accuracy`: The produced SR models should be accurate. Accuracy is measured in terms of the [R2 Score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html?highlight=r2_score#sklearn.metrics.r2_score) measured on the test set.
To account for the fact that methods achieving similar R2 Scores can be considered equivalently good, the R2 Score is rounded to the 3rd decimal (e.g., 0.8712 -> 0.871).
* `simplicity`: SR models should be simpler in order to stand a higher chance of being interpretable. As a loose metric for this, the number of components of the model (post simplification via `sympy`) is considered as measure of complexity, i.e., the opposite of simplicity.
We consider finding an accurate but less complex model to become increasingly hard with decreasing levels of complexity.
Thefefore, complexity is actually calculated as a function of `round(-log_5(s), 1)`, where `s` is the number of components in the (`sympy`-simplified) model. This translates to the figure below, such that, e.g., a model with 4 components is considered to be better than one with 5 components, while a model with 85 components is considered to be the same of one with 90 components.
* `property`: Each synthetic data set is prepared with respect to a certain property (e.g., presence of many irrelevant features or ability to re-discover the data-originating function).
The models generated by a method should reflect that property (e.g., respectively, make no use of irrelevant features or be symbolically equivalent to the data-originating function).
Since the data sets are private, so are their properties.
However, we can disclose that frequency of discovery of the data-originating function is one of the properties.

Given the aspects above, the participating methods will be evaluated as follows:
1. Each method is run 10 times, producing 10 models, for each data set.

1. For each data set, the `accuracy`, `simplicity`, and `property` of each SR method are measured. The first two are the same across each data set, while the latter is data set-specific.
For each aspect, the median of the 10 repetitions is considered.

2. Independently for each data set, the participating methods are ranked in terms of the four aspects. Higher rank = better performance.

2. Next, a data set-specific score is computed across the three aspects, by taking the harmonic mean of the respective ranks (i.e., `score_dataset = harmonic_mean(rank_accuracy, rank_simplicity, rank_property)`); this promotes methods producing models that have a good trade-off between the different aspects (e.g., an algorithm that produces most-accurate but very complex models will score worse than an algorithm that produces decently-accurate and not too complex models).

3. A final score is obtained by averaging across data set-specific scores: `final_score = mean_i(score_dataset_i)`.

The algorithm with highest final score wins this track of the competition.


# Winners


|    | algorithm     |   hmean_score |
|---:|:--------------|--------------:|
|  1 | QLattice      |          6.23 |
|  2 | pysr          |          5.26 |
|  3 | uDSR          |          5.67 |
|  4 | operon        |          4.38 |
|  5 | Bingo         |          4.32 |
|  6 | E2ET          |          2.74 |
|  7 | geneticengine |          2.54 |
|  8 | eql           |          1.33 |
|  9 | PS-Tree       |          0.85 |

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from pathlib import Path


In [ ]:
rdir = '../results_stage1/'
datadir = '../experiment/data/stage1/data/'

In [ ]:
import pdb
frames = []
i = 0
for f in Path(rdir).rglob('*.json'):
#     print(f)
    if 'gpzgd' in str(f):
        continue
    if 'seir' in str(f):
        continue
#     if 'easier' in str(f):
#         continue
    with open(f, 'r') as of:
        d = json.load(of)
    if 'symbolic_equivalence' in d.keys():
        d['solution'] = int(d['symbolic_equivalence']['equivalent'])
        d['pred_model'] = d['symbolic_equivalence']['pred_model']
        d['sym_diff'] = d['symbolic_equivalence']['sym_diff']
        d['sym_frac'] = d['symbolic_equivalence']['sym_frac']
        try:
            d['true_model'] = d['symbolic_equivalence']['true_model']
        except Exception as e:
            pdb.set_trace()
        d['filename'] = f
    d['dataset'] = d['dataset'].replace('_base','_easy')
    del d['params']
#     if d['algorithm'] == 'Bingo' and 'seirdR' in d['dataset'] and 'easy' in d['dataset']:
#         print(d['r2_test'])
#         pdb.set_trace()
    frames.append(d)
    i += 1
    
print('loaded',i,'results')
df = pd.DataFrame.from_records(frames)

# fix cutoff dataset names
df['dataset'] = df['dataset'].apply(lambda x: x+'ata' if x.endswith('_d') else x)

########################################
# metric cleanup
# df['r2_test'] = df['r2_test'].round(5)
df['mse_test'] = df['mse_test'].round(3)
df['simplicity'] = (df['simplicity']-df['simplicity'].min())/(df['simplicity'].max()-df['simplicity'].min())
########################################
# get dataset sizes
dataset_nsamples = {}
dataset_nfeatures = {}
    
for d in df.dataset.unique():
    filename = datadir+d+'.csv'
    if not os.path.exists(filename):
        filename = datadir+'seir/'+d+'.csv'
    tmp = pd.read_csv(filename)
    dataset_nsamples[d] = len(tmp)
    dataset_nfeatures[d] = tmp.shape[1]-1

ns = pd.DataFrame({'dataset':dataset_nsamples.keys(),
              'nsamples':dataset_nsamples.values(),
             })
nf = pd.DataFrame({'dataset':dataset_nfeatures.keys(),
              'nfeatures':dataset_nfeatures.values(),
             })
data = pd.merge(ns,nf,on='dataset')
df = df.merge(data,on='dataset')
df['nsize'] = df['nsamples'].apply(lambda x: '>10,000' if x>10000 else '>1000' if x>1000 else '<=1000')
df['fsize'] = df['nfeatures'].apply(lambda x: '>=1000' if x>=1000 else '>=100' if x>=100 else '<100')
########################################
# time transform
df['time_hr'] = df['time_time']/3600
df['time_mins'] = df['time_time']/60
########################################
# merge dataset names
df['dataset-full'] = df['dataset'].copy()
df['dataset'] = df['dataset'].apply(lambda x: '_'.join(x.split('_')[1:]))
df['task'] = df['dataset'].apply(lambda x: x.split('_')[0] if 'exact' not in x else '_'.join(x.split('_')[:2]))
df['difficulty'] = df['dataset'].apply(lambda x: x.split('_')[1] if 'exact' not in x else x.split('_')[2])
df['difficulty'] = df['difficulty'].apply(lambda x: {'easier':'0-easier','easy':'1-easy','medium':'2-medium','hard':'3-hard'}[x])
df['random_state'] = df['dataset-full'].apply(lambda x: x.split('_')[0])
# df.loc[df.difficulty=='base','difficulty'] = 'easy'
# df['difficulty'] = df['difficulty'].astype('category')
# df.head()


In [ ]:
df.loc[(df.algorithm=='Bingo') 
       & (df.task=='seirdR')
       & (df.difficulty=='easy')
      ]['r2_test']

In [ ]:
df.columns

In [ ]:
df.task.unique()

In [ ]:
METRICS = [
   'r2_test', 
   'feature_absence_score',
   'solution',
   'simplicity'
]

def task_property(d):
    if 'seir' in d or 'exact_formula' in d:
        return 'solution'
    elif 'localopt' in d or 'featureselection' in d:
        return 'feature_absence_score'
    else:
        return 'r2_test'

df['property'] = df['task'].apply(task_property)
df[['task','property']].drop_duplicates()

# check run completion

In [ ]:
df = df.drop_duplicates(subset=['dataset-full','algorithm'])
tmp = df.groupby(['task','difficulty','algorithm'])['random_state'].count().unstack()
display(
    tmp.fillna('DNF')
)
print(tmp.mean(),'runs completed on average')


# summarize scores  and calculate rankings

- mean of trials are taken as the metric score for each method on each dataset. 
- ranks are calculated on a per dataset basis, using this mean score. 
- ranks are calculated for `r2_test_rounded`, `simplicity`, and a third property depending on dataset. 
- the harmonic mean of the rankings over all problems is the method's final score. 

In [ ]:
df[['dataset','property']]
df['property_score'] = df.apply(lambda x: x[x['property']], axis=1)
mets = ['r2_test','simplicity','property_score']
rank_mets = []
df_sum = (df.groupby(['algorithm','task','difficulty','dataset'],as_index=False)
          [mets]
          .mean()
         )
n_algs = df.algorithm.nunique()
df_sum['r2_test'] = df_sum['r2_test'].round(3)
# df_sum['simplicity'] = df_sum['simplicity'].round(3)
for col in mets:
#     ascending = 'r2' not in col and 'simplicity' not in col and 'feature_absence' not in col
    ascending=False
    colname = col+'_rank' 
    df_sum[colname]=(n_algs-df_sum
                     .groupby('dataset')
                     [col]
                     .rank(ascending=ascending, 
                           method='dense')
                    ) 
    assert df_sum[colname].max() <= df_sum.algorithm.nunique()
    rank_mets.append(colname)

"""
compute harmonic mean
"""
from scipy import stats
# df_sum['hmean_score'] = df.groupby('dataset')[rank_mets].agg(stats.hmean)
for task in ['exact_formula']:
    display(df_sum.loc[df_sum.task.str.contains(task)].groupby(['task','difficulty','algorithm']).median())

# harmonic mean 

In [ ]:
from scipy import stats
# df_sum['hmean_score'] = 
import pdb
max_rank = df_sum.algorithm.nunique()
def nanhmean(x):
    x[x.isna()]=max_rank
    assert len(x.values)==1
    return stats.hmean(x.values[0])
    
hmean_score = df_sum.groupby(['dataset','algorithm'])[rank_mets].apply(nanhmean).rename('hmean_score') #.apply(lambda x: 0.0 if np.isnan(x) else x) #.apply(stats.hmean)
df_sumh = df_sum.merge(hmean_score, on=['dataset','algorithm'])


# calculate winners

In [ ]:
winners = df_sumh.groupby(['algorithm'])['hmean_score'].mean().sort_values(ascending=False)
order = winners.index 
winners.reset_index().round(2).to_markdown()

In [ ]:
winners.reset_index()

In [ ]:
df_sum.groupby(['dataset','algorithm'])[rank_mets].mean().unstack().mean()

In [ ]:
df_plt = df_sumh.melt(id_vars = ['dataset','algorithm','task','difficulty'])
# df_plt = df_sumh.melt(id_vars = ['algorithm','task'])
df_plt = df_plt.loc[df_plt.variable.isin(['hmean_score']+rank_mets)]
g = sns.catplot(
    kind='point',
    join=False,
    palette='flare_r',
    data=df_plt,
    y='algorithm',
    order=order,
    x='value',
    col='variable',
    col_order = ['hmean_score']+rank_mets,
#     col_wrap=3,
    sharex=False,
    aspect=0.65,
    height=6
)
g.set(xlabel='Better ->',ylabel='')
for k,ax in g.axes_dict.items():
    ax.grid(axis='y')
    if 'hmean' not in k:
        ttl = ax.get_title()[11:]
    else:
        ttl = 'Overall Score (Hamonic Mean)'
#         ttl = ax.get_title()[11:]
    ax.set_title(ttl.replace('_',' ').title())
# sns.despine(left=True,bottom=True)

In [ ]:
df_plt

# scores and rankings for each dataset

In [ ]:
df_tbl = df_sumh.melt(id_vars = ['dataset','algorithm','task','difficulty'],var_name='metric')
# df_sumh.groupby(['task','difficulty','algorithm'])[rank_mets+['hmean_score']].mean()
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for task, dfg in df_tbl.groupby('task'):
        task_score = task_property(task)
        print(10*'=',task,10*'=')
        task_metrics = ['r2_test','simplicity','property_score']
        task_metrics = [tm+'_rank' for tm in task_metrics] + ['hmean_score']
#         task_metrics.extend([tm+'_rank' for tm in task_metrics])
        dfp = dfg.loc[dfg.metric.isin(task_metrics)]
    #     dfg.loc[dfg.metric.str.contains('rank'),'metric'] = dfg.loc[dfg.metric.str.contains('rank'),'metric'].round(1)
        dfp.loc[dfp.metric=='property_score','metric'] = task_score
        dfp.loc[dfp.metric=='property_score_rank','metric'] = task_score+'_rank'
        rdr = [o for o in order if o in dfg['algorithm'].unique()]
        display(dfp.groupby(['task','difficulty','algorithm','metric'])['value'].median().unstack().round(1)) #[rdr].round(2))

#         task_metrics = [tm+'_rank' for tm in task_metrics]
#         dfp = dfg.loc[dfg.metric.isin(task_metrics)]
#         dfp.loc[dfp.metric=='property_score','metric'] = task_score
#         dfp.loc[dfp.metric=='property_score_rank','metric'] = task_score+'_rank'
#         display(dfp.groupby(['task','difficulty','metric','algorithm'])['value'].median().unstack()[rdr])
        print(40*'=')

In [ ]:
metr = ['r2_test_rank','simplicity_rank','property_score_rank','hmean_score']
(df_tbl.loc[df_tbl.metric.isin(metr)]
 .groupby(['task','metric','algorithm'])['value']
 .mean()
 .unstack()
 .round(1)[order]
 .fillna('DNF')
)

In [ ]:
(df_tbl.loc[df_tbl.metric=='hmean_score']
 .groupby(['task','difficulty','metric','algorithm'])['value']
 .mean()
 .unstack()
 .round(1)[order]
 .fillna('DNF')
)

# explore solutions

In [ ]:
import sympy as sp
# define local namespace dictionary
SEIR_VARS = {}
for s in ['dS', 'dE', 'dI', 'dR',
          'x_S', 'x_E', 'x_I', 'x_R', 'x_rzero', 'x_gamma', 'x_sigma', 'x_c', 'x_N', 'x_Aux']:
    SEIR_VARS.update({ s: sp.Symbol(s) })
        
EF_VARS = {f'x{i}':sp.Symbol(f'x{i}') for i in range(5)}
LD = dict(seir = SEIR_VARS, ef=EF_VARS)
frames = []
dfg = df.loc[df.task=='exact_formula']
for (task,difficulty),dfgpg in dfg.groupby(['task','difficulty']):
    print(40*'=')
    print(task,difficulty)
    print('true_model:') #,true_model)
    local_dict = LD['seir'] if 'seir' in task else LD['ef']
    true_model = sp.parse_expr(dfgpg['true_model'].values[0], local_dict=local_dict)
    display(true_model)
    print(40*'=')
    for (alg,task,difficulty),dfz in dfgpg.groupby(['algorithm','task','difficulty']):
        row = dfz.loc[dfz['r2_test'].idxmax()]
        if len(row['pred_model']) > 1000:
            continue
        print(20*'-')
        for c in ['algorithm','r2_test','simplicity','solution','dataset-full']: #,'filename']:
            print(c,':',row[c],',',end=' ') #,'symbolic_model','pred_model','true_model']])
        print('\n')
        pred_model = sp.parse_expr(row['pred_model'], local_dict=local_dict)
        orig_model = sp.parse_expr(row['symbolic_model'], local_dict=local_dict)
#         print(row['filename'])
        if str(pred_model)=='0':
            print('symbolic_model:')
            display(orig_model)
        else:
            print('pred_model:')
            display(pred_model)
            print('sym_diff',row['sym_diff'])
            print('sym_frac',row['sym_frac'])
            print('original model:',orig_model)
        print(20*'-')
    print(40*'=')

# plot individual task results 

In [ ]:
import pdb
metrics = ['r2_test','solution','feature_absence_score','simplicity']
rank_metrics = [m+'_rank' for m in metrics]
i = 0
for task,dft in df.groupby('task'):
#     task_metrics = ['mse_test_rank','simplicity_rank',task_property(task)+'_rank'] 
    task_metrics = ['r2_test','simplicity',task_property(task)] 
    df_plt = dft.melt(id_vars = ['dataset','algorithm','difficulty','random_state'])
    df_plt = df_plt.loc[df_plt.variable.isin(task_metrics)]
    
    if task == 'exact_formula':
        hue_order=['0-easier','1-easy','2-medium','3-hard']
    else:
        hue_order=['1-easy','2-medium','3-hard']
        
    g = sns.catplot(
        kind='point',
        join=False,
        palette='flare_r',
        data=df_plt,
        y='algorithm',
        order=order,
        x='value',
        col='variable',
        hue='difficulty',
        hue_order=hue_order,
        sharex=False,
#         aspect=0.65,
#         height=6
    )
    g.set(ylabel='', xlabel='better ->')
    

    for k,ax in g.axes_dict.items():
        ax.grid(axis='y')
        ax.set_title(ax.get_title()[11:].title())
#         if 'mse' in k:
#             ax.set_xscale('log')
        if k=='r2_test':
            x = df_plt.loc[df_plt.variable==k]['value']
            x_left = ax.get_xlim()[0]
            if x_left < x.quantile():
#             if x_bottom < -1:
                ax.set_xlim(x.quantile(.1), 1.0)
    ax = g.axes.flat[0]
    ax.text(s=task,
#             x=0, y=1,
            x=ax.get_xlim()[0],
            ha='right',
            y=ax.get_ylim()[1],
            fontsize='large',
            color='w',
            backgroundcolor='k'
           )

